In [ ]:
import sys; sys.path.insert(0, '../../')

In [2]:
import tensorflow as tf
from models.snn.evaluate import evaluate_model
from models.snn.factory import build_snn
from models.preprocessing import load_preprocessed

2025-12-15 21:49:06.325416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-15 21:49:06.325465: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-12-15 21:49:07.974321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-15 21:49:09.260263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-15 21:49:09.260377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: l

In [3]:
if os.getenv("CONTAINER"):
    print(tf.config.list_physical_devices("GPU"))

In [4]:
search_hidden = [100, 200, 300]
search_syn = [0.005, 0.01, 0.02]

In [5]:
best_score = -1
best_params = None

In [6]:
splits, class_names = load_preprocessed(path + 'data/processed/')
X_train, y_train = splits['train']
X_val, y_val = splits['val']
X_test, y_test = splits['test']

Loaded train: X=(9, 30720), y=(9, 10)
Loaded val: X=(3, 30720), y=(3, 10)
Loaded test: X=(2, 30720), y=(2, 10)
Loaded class names: ['benjamin', 'christian', 'felix', 'jonas', 'leon', 'mark', 'marvin', 'ohman', 'veronica', 'yannes']


In [7]:
for h in search_hidden:
    for s in search_syn:
        acc, history = evaluate_model(h, s, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, n_features=X_train.shape[1], n_classes=len(class_names))
        print(f"Tested hidden={h}, syn={s}, val_acc={acc:.3f}")

        if acc > best_score:
            best_score = acc
            best_params = (h, s)

|                     Building network (0%)                    | ETA:  --:--:--
|######               Building network (10%)                     | ETA: 0:00:00
|######               Building network (10%)                     | ETA: 0:00:01
|#####################Building network (40%)                     | ETA: 0:00:00


/home/raah/.local/share/virtualenvs/neuro-ai-9gzpYvh7/lib/python3.10/site-packages/nengo_dl/simulator.py:456: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(


|#####################Building network (50%)                     | ETA: 0:00:00
|#####################Building network (60%)                     | ETA: 0:00:00
|#####################Building network (60%)                     | ETA: 0:00:00
|#####################Building network (70%)#                    | ETA: 0:00:00
Build finished in 0:00:00
|#                         Optimizing graph                           | 0:00:00
|#             Optimizing graph: operator simplificaton               | 0:00:00
Optimizing graph: operator simplificaton finished in 0:00:00
|#                Optimizing graph: merging operators                 | 0:00:00
Optimizing graph: merging operators finished in 0:00:00
|#                Optimizing graph: ordering signals                  | 0:00:00
Optimizing graph: ordering signals finished in 0:00:00
|#                Optimizing graph: creating signals                  | 0:00:00
Optimizing graph: creating signals finished in 0:00:00
Optimization finished in 0:

2025-12-15 21:49:10.579687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-15 21:49:10.579856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-12-15 21:49:10.579925: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-12-15 21:49:10.579984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-12-15 21:49:10.580044: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

|##############Constructing graph: build stage (52%)             | ETA: 0:00:00
|##############Constructing graph: build stage (78%)             | ETA: 0:00:00
Constructing graph: build stage finished in 0:00:00
|     #                   Constructing graph                          | 0:00:00
Construction finished in 0:00:00


ValidationError: obj: <Node (unlabeled) at 0x7792235b49a0> is not an input Node (a nengo.Node with size_in==0), or is from a different network.

In [ ]:
print("Best:", best_params, "Acc:", best_score)

In [ ]:
with open(path + 'checkpoints/snn/.env', 'w') as f:
    f.write(f"HIDDEN_UNITS={best_params[0]}\n")
    f.write(f"SYNAPTIC_WEIGHT={best_params[1]}\n")